#### import the things

In [2]:
import tweepy
from datetime import datetime, timedelta, timezone
import pandas as pd
import json
import os
import random
import time

In [7]:
# import sys
# sys.path.append(os.path.join(os.path.dirname(sys.path[0]), 'scripts'))
import scripts.data_acquisition_tools as dat

%load_ext autoreload
%autoreload 2

In [9]:
dat.test_sys_things()

['C:\\Users\\smana\\Documents\\Springboard\\Capstone_Two\\data_wrangling', 'C:\\Users\\smana\\anaconda3\\envs\\nlp\\python38.zip', 'C:\\Users\\smana\\anaconda3\\envs\\nlp\\DLLs', 'C:\\Users\\smana\\anaconda3\\envs\\nlp\\lib', 'C:\\Users\\smana\\anaconda3\\envs\\nlp', '', 'C:\\Users\\smana\\AppData\\Roaming\\Python\\Python38\\site-packages', 'C:\\Users\\smana\\anaconda3\\envs\\nlp\\lib\\site-packages', 'C:\\Users\\smana\\anaconda3\\envs\\nlp\\lib\\site-packages\\win32', 'C:\\Users\\smana\\anaconda3\\envs\\nlp\\lib\\site-packages\\win32\\lib', 'C:\\Users\\smana\\anaconda3\\envs\\nlp\\lib\\site-packages\\Pythonwin', 'C:\\Users\\smana\\anaconda3\\envs\\nlp\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\smana\\.ipython', 'C:\\Users\\smana\\Documents\\Springboard\\Capstone_Two']


In [ ]:
import sys, os
sys.path.append(os.path.dirname(sys.path[0]))

In [3]:
sys.path.append(os.path.dirname(sys.path[0]))

In [6]:
import twitter_api_access

In [5]:
import sys
sys.path

['C:\\Users\\smana\\Documents\\Springboard\\Capstone_Two\\data_wrangling',
 'C:\\Users\\smana\\anaconda3\\envs\\nlp\\python38.zip',
 'C:\\Users\\smana\\anaconda3\\envs\\nlp\\DLLs',
 'C:\\Users\\smana\\anaconda3\\envs\\nlp\\lib',
 'C:\\Users\\smana\\anaconda3\\envs\\nlp',
 '',
 'C:\\Users\\smana\\AppData\\Roaming\\Python\\Python38\\site-packages',
 'C:\\Users\\smana\\anaconda3\\envs\\nlp\\lib\\site-packages',
 'C:\\Users\\smana\\anaconda3\\envs\\nlp\\lib\\site-packages\\win32',
 'C:\\Users\\smana\\anaconda3\\envs\\nlp\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\smana\\anaconda3\\envs\\nlp\\lib\\site-packages\\Pythonwin',
 'C:\\Users\\smana\\anaconda3\\envs\\nlp\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\smana\\.ipython',
 'C:\\Users\\smana\\Documents\\Springboard\\Capstone_Two']

In [17]:
os.path.dirname(sys.path[0])

'C:\\Users\\smana\\Documents\\Springboard\\Capstone Two'

In [2]:
import .scripts.data_acquisition_tools as dat

SyntaxError: invalid syntax (<ipython-input-2-4a394ac58c20>, line 1)

In [2]:
from src.data import data_acquisition_tools as dat
%load_ext autoreload
%autoreload 2

In [3]:
import data
save_loc = os.path.join(data.__path__[0],r"raw")

#### load hashtags from previous step

In [4]:
hashtags = pd.read_pickle(os.path.join(data.__path__[0], r'compiled\common_hashtags.pkl'))

In [5]:
hashtags

{'aids',
 'disclose',
 'endaids',
 'endhivepidemic',
 'endhivstigma',
 'hiv',
 'hivhometest',
 'hivstigma',
 'hivtestweek',
 'hivtreatment',
 'plhiv',
 'plwhiv',
 'prep',
 'sciencenotstigma',
 'uequalsu',
 'vaccinatethemostvulnerable',
 'vaccinatethemostvulneranle'}

In [6]:
#had to remove #disclose due to the recent DISCLOSE act from US congress
#removed the last one as I didn't realize initially it's a typo
hashtags.remove('disclose')
hashtags.remove('vaccinatethemostvulneranle')

In [7]:
#format the hashtags to add operators and hash symbol where necessary, as well as language filter
#need to remove keyword covid because otherwise it dominates the results
add_hash = ["#{}".format(h) if (h=='aids') or (h=='prep') else h for h in hashtags]
add_or = " OR ".join(add_hash)
query = "({}) -covid lang:en".format(add_or)
print(len(query))
print(query)

223
(uequalsu OR hivstigma OR #aids OR hivtreatment OR #prep OR hivhometest OR endaids OR sciencenotstigma OR endhivstigma OR plhiv OR hivtestweek OR plwhiv OR vaccinatethemostvulnerable OR hiv OR endhivepidemic) -covid lang:en


#### do a test search on the standard api (with generous rate limits) to make sure everything works

In [8]:
fname = os.path.join(save_loc, 'test_search.json')

In [9]:
test = dat.api_search(query, fname, n_items=500, return_dict=True)
del test['query']

In [10]:
test[0]['created_at']

'Fri Mar 05 19:19:19 +0000 2021'

In [11]:
test[499]['created_at']

'Fri Mar 05 17:50:54 +0000 2021'

In [12]:
langs = []
for key in test.keys():
    langs.append(test[key]['lang'])

from collections import Counter
lang_counts = Counter(langs)
print(lang_counts.most_common(5))

[('en', 500)]


I had some trouble in previous pulls with the lang command in the query not working properly, but this seems to have worked. 

#### pull tweets

In order to account for the way topics cluster on twitter (due to intra-community discussion of the same topic over a short period of time), as well as potential time zone biases, every 5000 tweets (10 pulls), I set the timestamp to go back 2 days and randomize start time by subtracting a random number of hours (up to 23).

In [13]:
#set parameters to begin collecting tweets that are at least 1 week old
random.seed(42)
h = random.randint(0,23)
last_dt, last_str = dat.timestamp_to_toDate((datetime.now(timezone.utc) - timedelta(days=7, hours=h)).strftime("%a %b %d %H:%M:%S %z %Y"))

n = 1
saved_params = (last_dt, last_str, n)
saved_params

(datetime.datetime(2021, 2, 25, 23, 24, 41, tzinfo=datetime.timezone.utc),
 '202102252324',
 1)

In [15]:
#collect 20,000 tweets
last_dt, last_str = saved_params[:2]

n = saved_params[2]
while n <= 40:
    if n % 10 == 0:
        h = random.randint(0,23)
        last_dt, last_str = dat.timestamp_to_toDate((last_dt - timedelta(days=2, hours=h)).strftime("%a %b %d %H:%M:%S %z %Y"))
        print("going back 2 days")
    try:
        print(last_dt, "n={}".format(n))
        fname = os.path.join(save_loc, 'main_search{}.json'.format(n))
        search = dat.api_advanced_search(query, fname, which_api='30day', todate=last_str, n_items=500, return_dict=True)
        last_dt, last_str = dat.timestamp_to_toDate(search[499]['created_at'])    
        n += 1
    except Exception as e:
        saved_params = last_dt, last_str, n
        print(saved_params)
        print("exception = ", e)
        print("...sleeping...")
        time.sleep(60)
        continue

2021-02-25 16:54:28+00:00 n=7
2021-02-25 15:44:03+00:00 n=8
2021-02-25 14:39:34+00:00 n=9
going back 2 days
2021-02-23 10:34:01+00:00 n=10
2021-02-23 08:54:53+00:00 n=11
2021-02-23 06:55:06+00:00 n=12
2021-02-23 04:11:17+00:00 n=13
(datetime.datetime(2021, 2, 23, 4, 11, 17, tzinfo=datetime.timezone.utc), '202102230411', 13)
exception =  {'message': 'Exceeded rate limit', 'sent': '2021-03-05T19:27:12+00:00', 'transactionId': '0053f62600350551'}
...sleeping...
2021-02-23 04:11:17+00:00 n=13
2021-02-23 01:39:17+00:00 n=14
2021-02-22 23:20:45+00:00 n=15
2021-02-22 21:48:41+00:00 n=16
2021-02-22 20:22:22+00:00 n=17
2021-02-22 19:12:16+00:00 n=18
2021-02-22 18:05:57+00:00 n=19
(datetime.datetime(2021, 2, 22, 18, 5, 57, tzinfo=datetime.timezone.utc), '202102221805', 19)
exception =  {'message': 'Exceeded rate limit', 'sent': '2021-03-05T19:28:32+00:00', 'transactionId': '005668d500996756'}
...sleeping...
2021-02-22 18:05:57+00:00 n=19
going back 2 days
2021-02-20 17:06:29+00:00 n=20
2021-02-2